In [1]:
pip install kagglehub

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("netflix-inc/netflix-prize-data")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\ivan\.cache\kagglehub\datasets\netflix-inc\netflix-prize-data\versions\2


In [6]:
import pandas as pd
import re
from unidecode import unidecode
from glob import glob

In [8]:
# Función para limpiar texto
def limpiar_texto(texto):
    if isinstance(texto, str):
        texto = unidecode(texto)  # Remueve tildes y caracteres especiales
        texto = re.sub(r"[^\w\s]", "", texto)  # Elimina signos de puntuación
        texto = re.sub(r"\s+", " ", texto).strip()  # Espacios extra
    return texto

In [11]:
# Procesar archivos de ratings
# Cambiar la ruta dependiendo del Path que de en el segundo bloque de codigo
rating_files = glob("C:/Users/ivan/.cache/kagglehub/datasets/netflix-inc/netflix-prize-data/versions/2/combined_data_*.txt")

data = []

for file in rating_files:
    with open(file, "r", encoding="utf-8") as f:
        movie_id = None
        for line in f:
            line = line.strip()
            if line.endswith(":"):  # Identificar ID de película
                movie_id = int(line[:-1])
            else:
                try:
                    user_id, rating, date = line.split(",")
                    data.append([movie_id, int(user_id), int(rating), date])
                except ValueError:
                    print(f"Error al procesar la línea: {line}")

# Convertir en DataFrame
df_ratings = pd.DataFrame(data, columns=["movie_id", "user_id", "rating", "date"])
df_ratings.to_csv("netflix_ratings_clean.csv", index=False)

print("Archivo de calificaciones procesado y guardado.")

Archivo de calificaciones procesado y guardado.


In [13]:
print(df_ratings.head(10))
print(df_ratings.shape)

   movie_id  user_id  rating        date
0         1  1488844       3  2005-09-06
1         1   822109       5  2005-05-13
2         1   885013       4  2005-10-19
3         1    30878       4  2005-12-26
4         1   823519       3  2004-05-03
5         1   893988       3  2005-11-17
6         1   124105       4  2004-08-05
7         1  1248029       3  2004-04-22
8         1  1842128       4  2004-05-09
9         1  2238063       3  2005-05-11
(100480507, 4)


In [15]:
# Cargar archivo de películas
df_movies = pd.read_csv("C:/Users/ivan/.cache/kagglehub/datasets/netflix-inc/netflix-prize-data/versions/2/movie_titles.csv", 
                        encoding="latin1", header=None, names=["movie_id", "year", "title"], delimiter=",", on_bad_lines='skip')

df_movies['year'] = df_movies['year'].fillna(0)
df_movies['year'] = df_movies['year'].astype(int)

# Limpiar los títulos de películas
df_movies["title"] = df_movies["title"].apply(limpiar_texto)

# Guardar el archivo limpio
df_movies.to_csv("movie_titles_clean.csv", index=False)

print("Archivo de títulos de películas procesado y guardado.")

Archivo de títulos de películas procesado y guardado.


In [17]:
print(df_movies.head(10))
print(df_movies.shape)

   movie_id  year                       title
0         1  2003             Dinosaur Planet
1         2  2004  Isle of Man TT 2004 Review
2         3  1997                   Character
3         4  1994   Paula Abduls Get Up Dance
4         5  2004    The Rise and Fall of ECW
5         6  1997                        Sick
6         7  1992                       8 Man
7         8  2004         What the Do We Know
8         9  1991     Class of Nuke Em High 2
9        10  2001                     Fighter
(17434, 3)


In [21]:
# Unir df_ratings con df_movies utilizando 'movie_id'
df_final = pd.merge(df_ratings, df_movies[['movie_id', 'title']], on='movie_id', how='inner')

# Seleccionar solo las columnas requeridas: 'movie_id', 'title', 'user_id', 'rating'
df_final = df_final[['movie_id', 'title', 'user_id', 'rating']]

In [22]:
# Mostrar el DataFrame final
print(df_final.head(10))
print(df_final.shape)

   movie_id            title  user_id  rating
0         1  Dinosaur Planet  1488844       3
1         1  Dinosaur Planet   822109       5
2         1  Dinosaur Planet   885013       4
3         1  Dinosaur Planet    30878       4
4         1  Dinosaur Planet   823519       3
5         1  Dinosaur Planet   893988       3
6         1  Dinosaur Planet   124105       4
7         1  Dinosaur Planet  1248029       3
8         1  Dinosaur Planet  1842128       4
9         1  Dinosaur Planet  2238063       3
(98985951, 4)


In [25]:
df_final.to_csv("netflix_ratings_with_titles.csv", index=False)